Based on https://albumentations.ai/docs/examples/example/

In [14]:
import os

path2original = "./data/original"
path2augmented = "./data/augmented"

# Copying originals
from distutils.dir_util import copy_tree
fls = copy_tree(path2original, path2augmented)

In [15]:
def getFieldPath(field):
    return path2augmented + "/" + field

def getImagePath(field, imageName):
    return getFieldPath(field) + "/" + imageName

def getImageNames(field):
    return sorted(os.listdir(path2augmented + "/" + field))

fields = sorted(os.listdir(path2original))
fields, len(fields)

(['1112', '1461', '1463', '1464', '1470', '1471'], 6)

In [16]:
import random

import cv2
from matplotlib import pyplot as plt

import albumentations as A

def visualize(image):
    plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(image)

In [18]:
import itertools

def transform(img, flip, rotation):
    if flip != None:
        img = cv2.flip(img, flip)
    if rotation != None:
        img = cv2.rotate(img, rotation)
    return img

def generateBasicAugmentations(fields):
    
    flips = [0, 1, -1]
    flipToIdMap = {k: v for v, k in enumerate(flips)}
    rotations = [None, cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_180, cv2.ROTATE_90_COUNTERCLOCKWISE]
    rotationToIdMap = {k: v for v, k in enumerate(rotations)}
    
    transform_combs = list(itertools.product(flips, rotations))
    
    for field in fields:
        imageNames = getImageNames(field)
        
        for imageName in imageNames:
            path = getImagePath(field, imageName)
            image = cv2.imread(path)
            #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            for flip, rotation in transform_combs:
                flipId = flipToIdMap[flip]
                rotationId = rotationToIdMap[rotation]
                
                new_field = f'{field}_{flipId}_{rotationId}'
                new_field_path = getFieldPath(new_field)
                
                if not os.path.exists(new_field_path):
                    os.makedirs(new_field_path)

                augmented_image = transform(image, flip, rotation)
                augmented_image_path = getImagePath(new_field, imageName)
                cv2.imwrite(augmented_image_path, augmented_image)

In [19]:
generateBasicAugmentations(fields)

Transforms https://albumentations.ai/docs/examples/example_kaggle_salt/#non-destructive-transformations-dehidral-group-d4